In [1]:
import pickle

from dataset import NewsDataset
from DistilbertModel import DistilBertForSequenceClassification

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
import pandas as pd
from IPython.display import display, HTML
import numpy as np

In [2]:
config = DistilBertConfig()
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# distilbert = DistilBertForSequenceClassification(config, num_labels=2)

In [3]:
distilbert = DistilBertForSequenceClassification(config=config)#.from_pretrained(None,config=config,state_dict='distilbert_cad_final1.pt')

In [4]:
criterion = nn.CrossEntropyLoss()

In [5]:
batch_size = 1

In [6]:
path = './distilbert_cad_final1.pt'

if torch.cuda.is_available():
    distilbert.load_state_dict(
        torch.load(path)
    )
else:
    distilbert.load_state_dict(
        torch.load(path, map_location=torch.device('cpu'))
    )
    
# # with open('../label_encoder.sklrn', 'rb') as f:
# #     le = pickle.load(f)

In [7]:
from saliency_interpreter import SaliencyInterpreter

In [9]:
SaliencyInterpreter(model=distilbert, criterion=criterion, tokenizer=tokenizer)._get_gradients()

In [25]:
# distilbert = DistilBertForSequenceClassification.from_pretrained('../tcav/bertcheckpoints/distilbert/distilbert_cad_final1.h5',output_attentions=True, from_tf=True)

In [45]:
# np.array([[0]*512]).size()

In [41]:
# predictions=distilbert(input_ids=torch.tensor([[0]*512]),return_dict=True,output_hidden_states=True)

In [46]:
# vars(predictions)

In [10]:
test_df = pd.read_csv('../tcav2/data/CAD_test.csv')

In [11]:
texts = list(test_df['TEXT'])

In [17]:
# len(texts)

1455

In [27]:
all_instances = []
texts=['Patient was a pleasant male in his early 40s who was given plavix and aspirin for CAD and required cabg', 'Patient is healthy']
for text in texts:
    test_example = [
        [text], 
        [""]
    ]

    test_dataset = NewsDataset(
        data_list=test_example,
        tokenizer=tokenizer,
        max_length=config.max_position_embeddings, 
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    integrated_grad = IntegratedGradient(
        distilbert, 
        criterion, 
        tokenizer, 
        show_progress=True,
        encoder="bert"
    )
    instances = integrated_grad.saliency_interpret(test_dataloader)
    all_instances.append(instances)

100%|██████████| 1/1 [00:36<00:00, 36.02s/it]


In [28]:
coloder_string = integrated_grad.colorize(instances[0])
display(HTML(coloder_string))

In [30]:
grads_df = pd.DataFrame([instance[0] for instance in all_instances])#.to_csv('all_test_grads.csv')

In [2]:
def unravel(lst):
    return [i for j in lst for i in j]

In [1]:
import pandas as pd

# grads_df = pd.read_csv('all_test_grads2.csv')
grads_df = pd.read_csv('all_train_grads2.csv')

In [3]:
grads_df['tokens']=grads_df['tokens'].apply(eval)

In [4]:
grads_df['grad'] = grads_df['grad'].apply(eval)

In [5]:
all_positive_grads = pd.DataFrame(  list(zip( unravel(grads_df[grads_df['label']==1]['tokens']),  unravel(grads_df[grads_df['label']==1]['grad']) )) , columns=['tokens', 'grad']  )

In [6]:
all_negative_grads = pd.DataFrame(  list(zip( unravel(grads_df[grads_df['label']==0]['tokens']),  unravel(grads_df[grads_df['label']==0]['grad']) )) , columns=['tokens', 'grad']  )

In [7]:
# all_negative_grads

In [8]:
word =''
score=0
words=[]
scores=[]
for i in reversed(range(len(all_positive_grads))):
    token=all_positive_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_positive_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        words.append(word)
        score = max(score,all_positive_grads['grad'].iloc[i] )
        scores.append(score)
        word=''
        score=0

In [9]:
tokens_grouped = pd.DataFrame(list(zip(words, scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
positive_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
positive_df = positive_df[positive_df['word_count']>5]
print(positive_df.head(25).to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrr}
\toprule
           word &  grad\_mean &  word\_count \\
\midrule
         cabgx1 &      0.127 &          14 \\
         cabgx3 &      0.110 &          45 \\
         cabgx5 &      0.107 &           7 \\
         cabgx4 &      0.094 &          41 \\
         cabgx2 &      0.086 &          21 \\
           cabg &      0.083 &        2210 \\
          [SEP] &      0.067 &        9364 \\
            cad &      0.063 &        1344 \\
 bronchodilator &      0.054 &           6 \\
            rca &      0.052 &         655 \\
         artery &      0.039 &         703 \\
     integrelin &      0.035 &         105 \\
           simv &      0.034 &         351 \\
     fasciotomy &      0.033 &           6 \\
          wires &      0.030 &        1200 \\
           pvcs &      0.029 &         310 \\
            lad &      0.028 &         981 \\
    thoracotomy &      0.028 &          31 \\
            ace &      0.027 &         365 \\
        insulin &      0.027 &        2

In [10]:
word =''
score=0
negative_words=[]
negative_scores=[]
for i in reversed(range(len(all_negative_grads))):
    token=all_negative_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_negative_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        negative_words.append(word)
        score = max(score,all_negative_grads['grad'].iloc[i] )
        negative_scores.append(score)
        word=''
        score=0

In [11]:
tokens_grouped = pd.DataFrame(list(zip(negative_words, negative_scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
negative_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
negative_df = negative_df[negative_df['word_count']>5]
print(negative_df.head(25).to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrr}
\toprule
                word &  grad\_mean &  word\_count \\
\midrule
                 cad &      0.094 &          76 \\
               [SEP] &      0.081 &        6819 \\
                 lad &      0.075 &           7 \\
           narrative &      0.069 &          37 \\
                cabg &      0.068 &          10 \\
                 npn &      0.064 &         762 \\
           cadaveric &      0.057 &           7 \\
                 pvc &      0.047 &         480 \\
                pvcs &      0.042 &         166 \\
                 ccu &      0.040 &         306 \\
                simv &      0.037 &          72 \\
         prostectomy &      0.036 &           6 \\
         thorocotomy &      0.036 &           6 \\
              pedial &      0.035 &          20 \\
               tsicu &      0.035 &         516 \\
   gastrojejunostomy &      0.033 &           7 \\
           lobectomy &      0.032 &          65 \\
                sicu &      0.032 &      

In [93]:
list(means

['grad']

In [72]:
pd.DataFrame(list(zip(words, scores)), columns=['words', 'grad'])

,words,grad
0,',0.028707
1,a,0.051793
2,i,0.001401
3,s,0.000330
4,e,0.005617
...,...,...
117397,L,0.007958
117398,C,0.003365
117399,[,0.001415
117400,',0.005063


In [57]:
import pandas as pd
pd.DataFrame(list(zip(instances[0]['tokens'], instances[0]['grad'])))

IndexError: list index out of range

In [20]:
instances

[{'tokens': ['[CLS]',
   "'",
   'patient',
   'was',
   'a',
   'pleasant',
   'male',
   'in',
   'his',
   'early',
   '40',
   '##s',
   'who',
   'was',
   'given',
   'pl',
   '##avi',
   '##x',
   'and',
   'as',
   '##pi',
   '##rin',
   'for',
   'cad',
   'and',
   'required',
   'cab',
   '##g',
   "'",
   '[SEP]',
   'transform',
   '##er',
   '-',
   'based',
   'models',
   'have',
   'taken',
   'a',
   'leading',
   'role',
   'in',
   'nl',
   '##p',
   'today',
   '.',
   '[SEP]'],
  'grad': [0.0024040760472416878,
   0.011202486231923103,
   0.013759119436144829,
   0.002332319738343358,
   0.011373970657587051,
   0.02735648676753044,
   0.015535157173871994,
   0.030777039006352425,
   0.008332711644470692,
   0.013340604491531849,
   0.0001066805562004447,
   0.0027349493466317654,
   0.004292184952646494,
   0.00442542415112257,
   0.007150531280785799,
   0.0028611449524760246,
   0.015010221861302853,
   0.018782952800393105,
   0.01669878326356411,
   0.009789